In [1]:
!pip install git+https://github.com/PatBall1/detectree2.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/PatBall1/detectree2.git to /tmp/pip-req-build-15z_nyyq
  Running command git clone -q https://github.com/PatBall1/detectree2.git /tmp/pip-req-build-15z_nyyq
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-install-sejmirz5/detectron2_4093e78330154b8e8261bae078af57c5
  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-install-sejmirz5/detectron2_4093e78330154b8e8261bae078af57c5
     |████████████████████████████████| 274 kB 4.4 MB/s 
     |████████████████████████████████| 400 kB 31.6 MB/s 
     |████████████████████████████████| 48 kB 4.6 MB/s 
     |████████████████████████████████| 2.1 MB 28.4 MB/s 
     |████████████████████████████████| 1.0 MB 62.0 MB/s 
     |████████████████████████████████| 19.3 MB 1.1 MB/s 
     |████████████████████████████████| 16.7 MB 36.4 MB/s 
     |███████████████████

In [2]:
from detectree2.preprocessing.tiling import tile_data_reduced, getFeatures
import rasterio
import geopandas as gpd
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.7/dist-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


Mounted at /content/drive


In [3]:
img_path = "/content/drive/Shareddrives/detectreeRGB/paracou/rgb/Paracou_RGB_2016_10cm.tif"
crown_path = "/content/drive/Shareddrives/detectree2/data/Paracou/crowns/UpdatedCrowns4.gpkg"
out_dir = "./"
# Read in the tiff file
data = rasterio.open(img_path)
#data = img_data(data)
# Read in crowns
crowns = gpd.read_file(crown_path)
crowns = crowns.to_crs(data.crs.data)
print(
    "shape =",
    data.shape,
    ",",
    data.bounds,
    "and number of bands =",
    data.count,
    ", crs =",
    data.crs,
)
#print(crowns)

#overlapping_crowns = sjoin(crowns, geo_central, how="inner")
#overlapping_crowns = overlapping_crowns.explode(index_parts=True)
#print("Overlapping crowns:", overlapping_crowns)
buffer = 20
tile_width = 80
tile_height = 80
threshold = 60

shape = (40000, 35000) , BoundingBox(left=284362.126, bottom=580621.538, right=287862.126, top=584621.538) and number of bands = 4 , crs = EPSG:2972


/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [4]:
tile_data_reduced(data, out_dir, buffer, tile_width, tile_height, crowns, threshold)

Streaming output truncated to the last 5000 lines.
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/detectree2/preprocessing/tiling.py:214: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:2972
Right CRS: +init=epsg:4326 +no_defs +type=crs

  overlapping_crowns = sjoin(crowns, geo_central, how="inner")
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is

In [ ]:
!rm *.tif
!rm *.geojson
!rm *.png

In [ ]:
#from detectree2.models.train import get_tree_dicts
import os
import json
import cv2
import numpy as np
from detectron2.structures import BoxMode


def get_tree_dicts(directory, classes=None):
    #filepath = '/content/drive/MyDrive/forestseg/paracou_data/Panayiotis_Outputs/220303_AllSpLabelled.gpkg'
    #datagpd = gpd.read_file(filepath)
    #List_Genus = datagpd.Genus_Species.to_list()
    #Genus_Species_UniqueList = list(set(List_Genus))

    #list_of_classes = ['Pradosia_cochlearia','Eperua_falcata','Dicorynia_guianensis','Eschweilera_sagotiana','Eperua_grandiflora','Symphonia_sp.1','Sextonia_rubra','Vouacapoua_americana','Sterculia_pruriens','Tapura_capitulifera','Pouteria_eugeniifolia','Recordoxylon_speciosum','Chrysophyllum_prieurii','Platonia_insignis','Chrysophyllum_pomiferum','Parkia_nitida','Goupia_glabra','Carapa_surinamensis','Licania_alba','Bocoa_prouacensis','Lueheopsis_rugosa']
    classes = list_of_classes
    #classes = Genus_Species_UniqueList #['tree'] # genus_species list
    dataset_dicts = []
    for filename in [file for file in os.listdir(directory) if file.endswith('.geojson')]:
        json_file = os.path.join(directory, filename)
        with open(json_file) as f:
            img_anns = json.load(f)
        
        
        
        record = {}
        
        filename = os.path.join(directory, img_anns["imagePath"])
        # Make sure we have the correct height and width
        height, width = cv2.imread(filename).shape[:2]

        record["file_name"] = filename
        record["height"] = height
        record["width"] = width
        record["image_id"] = filename[0:400]
        print(filename[0:400])
            
        

        objs = []
        for features in img_anns['features']:
            anno = features['geometry']
            #pdb.set_trace()
            #GenusSpecies = features['properties']['Genus_Species']
            #print("##### HERE IS AN ANNO #####", anno)...weirdly sometimes (but not always) have to make 1000 into a np.array
            px = [a[0] for a in anno['coordinates'][0]]
            py = [np.array(height) - a[1] for a in anno['coordinates'][0]]
            #print("### HERE IS PY ###", py)
            poly = [(x, y) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]
            #print("#### HERE ARE SOME POLYS #####", poly)
            obj = {
                   "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                   "bbox_mode": BoxMode.XYXY_ABS,
                   "segmentation": [poly],
                   "category_id": classes.index(features['properties']['PlotNum']),  #id
                   "iscrowd": 0
                   }
            #pdb.set_trace()
            objs.append(obj)
            # print("#### HERE IS OBJS #####", objs)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

  

In [ ]:
variable = 'PlotOrg'
crowns[variable].unique().tolist()

['CIRAD', 'CNES', 'INRA']

In [ ]:
#from detectree2.models.train import get_tree_dicts
dataset_dicts = get_tree_dicts(out_dir)
dataset_dicts

././tile_286762_583741.png


KeyError: ignored

In [ ]:
# Let's look at our training image and annos for our geojson sepilok dataset!!!
from detectron2.data import DatasetCatalog, MetadataCatalog
import random
from PIL import Image

for d in ["train1"]:
    DatasetCatalog.register("trees_" + d, lambda d=d: get_tree_dicts('./'))
    MetadataCatalog.get("trees_" + d).set(thing_classes=['tree'])

trees_metadata = MetadataCatalog.get("trees_train1")
from detectron2.utils.visualizer import Visualizer
dataset_dicts = get_tree_dicts("./")
for d in dataset_dicts:
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=trees_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    image = cv2.cvtColor(out.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB)
    #display(Image.fromarray(img))
    display(Image.fromarray(image))

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
for d in dataset_dicts:
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=trees_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    image = cv2.cvtColor(out.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB)
    #display(Image.fromarray(img))
    display(Image.fromarray(image))

NameError: ignored